In [ ]:

import torch
from torch.utils.data import DataLoader
from state_quantization.dataset import load_dataset
from state_quantization.dataset import DynamicsModelDataset
from state_quantization.forcasting_models import LSTMForcasting
from state_quantization.quantization_models import DiscAutoEncoder
from state_quantization.forcasting_quantization_models import ForcastingQuant, EmbeddedAEForcastingQuant, ForcastingQuantInferenceWrapper, SingleEmbeddedAEForcastingQuant
from state_quantization.trainer import ForcastingQuantTrainer
from torch.optim.lr_scheduler import MultiStepLR
from state_quantization.train import train_model, test_step
from state_quantization.eval import eval_model, compare_models
import os

In [ ]:
# parameters

bits = 20
use_cuda = torch.cuda.is_available()
model_path = f'tmp/state_quantization/model_aeq-{bits}bits6'
untrained_model_path = f'tmp/state_quantization/untrained_model_aeq-{bits}bits'
log_dir='runs'

In [ ]:


os.makedirs(os.path.dirname(model_path), exist_ok=True)
os.makedirs(os.path.dirname(untrained_model_path), exist_ok=True)
device = torch.device("cuda:0" if use_cuda else "cpu")
# print(f"Using Device: {device}")
# torch.backends.cudnn.benchmark = True

## Load Dataset

In [ ]:
dataset_input_key = 'merged_input'
dataset_output_key = 'merged_output'
dataset_file_path = 'tmp/ib-out/ib-samples-la.npy'
normalized_data_params_save_path = 'tmp/transformer/NormalizeInputConfigs.pkl'
dataset_device = 'cpu'
y_indexes = [4, 6]

train_dataset, val_dataset = load_dataset(file_path=dataset_file_path, input_key=dataset_input_key,
                                          output_key=dataset_output_key, dataset_class=DynamicsModelDataset,
                                          normalize=True, device=dataset_device, y_clip_range=y_indexes,
                                          normalized_input_data_params_save_path=normalized_data_params_save_path)

batch_size = 8000
params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 0,
          'drop_last': True,
          'pin_memory': not train_dataset.x.is_cuda}
print(params)
train_loader = DataLoader(train_dataset, **params)
val_loader = DataLoader(val_dataset, **params)
train_dataset.y[0]

## Define model

In [ ]:


num_of_features = train_dataset.get_features_size()
seq_len = train_dataset.get_seq_len()
hidden_size = 20
out_size = train_dataset.get_output_feature_size()
print(f'Out Size:{out_size}')
look_ahead = train_dataset.get_look_ahead_size()
n_layers = 1
dropout = 0.1

forcasting_model = LSTMForcasting(features=num_of_features, hidden_size=hidden_size, out_size=out_size, seq_len=seq_len,
                                  look_ahead=look_ahead, dropout=dropout, n_layers=n_layers)

disc_autoencoder_input_size = hidden_size * 2
bottleneck_size = bits
ae_dropout = 0
disc_autoencoder = DiscAutoEncoder(input_size=disc_autoencoder_input_size, bottleneck_size=bottleneck_size,
                                   dropout=ae_dropout)

model = ForcastingQuant(forcasting_model=forcasting_model, autoencoder_quant_model=disc_autoencoder).to(device=device)
eval_model = EmbeddedAEForcastingQuant(model=model).to(device=device)
torch.save(ForcastingQuantInferenceWrapper(model), untrained_model_path)
load_to_gpu = model.is_cuda() and not train_dataset.x.is_cuda
print(load_to_gpu)



## Define Trainer

In [ ]:
gamma = 0.1

forecasting_learning_rate = 1e-3
autoencoder_learning_rate = 1e-3

forecasting_lr_milestones = [35]
autoencoder_lr_milestones = [100, 150]
forecasting_optimizer = torch.optim.Adam(model.forcasting_model.parameters(),
                                         lr=forecasting_learning_rate)
autoencoder_optimizer = torch.optim.Adam(model.autoencoder_quant_model.parameters(),
                                         lr=autoencoder_learning_rate)
forecasting_lr_scheduler = MultiStepLR(forecasting_optimizer, milestones=forecasting_lr_milestones, gamma=gamma)
autoencoder_lr_scheduler = MultiStepLR(autoencoder_optimizer, milestones=autoencoder_lr_milestones, gamma=gamma)
n_epochs = 200

trainer = ForcastingQuantTrainer(forcasting_quant_model=model, train_loader=train_loader, test_loader=val_loader,
                                 load_to_gpu=load_to_gpu, forecasting_optimizer=forecasting_optimizer,
                                 forecasting_lr_scheduler=forecasting_lr_scheduler,
                                 autoencoder_lr_scheduler=autoencoder_lr_scheduler,
                                 autoencoder_optimizer=autoencoder_optimizer,autoencoder_training_start=50,additional_eval_model=eval_model, log_dir=log_dir)



## Train Model

In [ ]:
trainer.train(n_epochs=n_epochs)

## Save Model

In [ ]:
emb_model_path = f'{model_path}emb'


In [ ]:
model = ForcastingQuantInferenceWrapper(model)

In [ ]:
torch.save(eval_model, emb_model_path)
torch.save(model, model_path)
torch.cuda.empty_cache()
del model